## Exercise: ATLAS $H \rightarrow 4 \ell $  
Measurement of the Higgs boson mass in the $H \rightarrow ZZ^* \rightarrow 4 \ell$, and $H \rightarrow \gamma \gamma$ channels with $\sqrt{s} = 13$ TeV $pp$ collisions using the ATLAS detector

#### PART 1: Model, Fit, Plot

Events have been selected given the presence of 4 leptons with invariant mass in the 110-135 GeV range. Each line corresponds to one single event. The observable is the invariant mass of the 4-leptons system.

In [1]:
import ROOT

In [2]:
# declare the observable: invariant mass of the 4-leptons system
m = ROOT.RooRealVar("m", "m(4 \\ell) [\\mathrm{GeV}]", 110, 135)
m.setBins(10)  # set number of bins to 10 to match the plot from original paper

# import the dataset as RooFit unbinned dataset (RooDataSet)
data1 = ROOT.RooDataSet.read("higgs_4l.dat", ROOT.RooArgList(m), "v")  
data = data1.reduce(ROOT.RooArgSet(m))  # remove "blindState" variable from the dataset

data.Print("v")

[#1] INFO:DataHandling -- RooDataSet::read: reading file higgs_4l.dat
[#1] INFO:DataHandling -- RooDataSet::read: read 123 events (ignored 0 out of range events)
DataStore dataset (higgs_4l.dat)
  Contains 123 entries
  Observables: 
    1)  m = 133.437  L(110 - 135) B(10)  "m(4 \ell) [\mathrm{GeV}]"


In [3]:
# visualize the data
c = ROOT.TCanvas()
mframe = m.frame()
mframe.SetTitle("Invariant Mass Distribution")
data.plotOn(mframe)
mframe.Draw()
c.Draw()

In [4]:
#### Define a model for the invariant mass distribution ####

# signal model: Crystal Ball shape distribution for H -> 4l peak
# define the mean of CB which is the Higgs mass
mass = ROOT.RooRealVar("mass", "higgs mass", 130, 110, 150)
# define the width of CB which is a constant
width = ROOT.RooRealVar("width", "higgs width", 4.1/2.35)
# define the variable alpha which is a constant
alpha = ROOT.RooRealVar("alpha", "width of CB", 0.6)
# define the power law exponent which is a constant
n = ROOT.RooRealVar("n", "power-law exponent", 20)

# build the signal model
smodel = ROOT.RooCBShape("smodel", "signal model", m, mass, width, alpha, n)

In [5]:
# background model: a polynomial of degree 2
a1 = ROOT.RooRealVar("a1", "a1 of background", -160, -200, -100)
a2 = ROOT.RooRealVar("a2", "a2 of background", 2.7, 2, 4)

# build the background model
bmodel = ROOT.RooPolynomial("bmodel", "background model", m, ROOT.RooArgList(a1, a2))

Define the expected yields for the different components of the spectrum (nsignal, nbackground).

In [6]:
# create the composite model (rate + shape -> extended likelihood -> marked poisson model)
nsig = ROOT.RooRealVar("nsig", "signal yield", 100, 0, 500)  
nbkg = ROOT.RooRealVar("nbkg", "background yield", 100, 0, 500)
model = ROOT.RooAddPdf("model", "signal + background model", ROOT.RooArgList(smodel, bmodel), ROOT.RooArgList(nsig, nbkg))

In [7]:
# Fit the model to the data using a maximum likelihood fit
fit_results = model.fitTo(data, ROOT.RooFit.Save())

# Plot data and model
c1 = ROOT.TCanvas("c1", "c1")
mframe1 = m.frame()
mframe1.SetTitle("Invariant Mass Distribution")

data.plotOn(mframe1, ROOT.RooFit.Name("Data"))  
model.plotOn(mframe1, ROOT.RooFit.Name("Fit"), ROOT.RooFit.LineColor(ROOT.kRed+1)) 
model.plotOn(mframe1, ROOT.RooFit.Components("bmodel"), ROOT.RooFit.Name("Background"),
             ROOT.RooFit.LineColor(ROOT.kBlack), ROOT.RooFit.LineStyle(ROOT.kDashed))       

mframe1.Draw()

legend = ROOT.TLegend(0.65, 0.7, 0.85, 0.85)
legend.AddEntry(mframe1.findObject("Data"), "Data", "lep")
legend.AddEntry(mframe1.findObject("Fit"), "Fit", "l")
legend.AddEntry(mframe1.findObject("Background"), "Background", "l")

legend.Draw()

label = ROOT.TLatex()
label.SetNDC() # normalised coordinates
label.SetTextSize(0.03)
label.DrawLatex(0.12, 0.70, "H \\rightarrow Z Z^{*} \\rightarrow 4 \\ell")
label.DrawLatex(0.12, 0.75, "\\sqrt{s} = 13 \\mathrm{TeV}, 36.1 \\mathrm{fb}^{-1}")

label.Draw()

c1.Draw()

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -96.0244009706939181
Edm   = 0.000399674072676061603
Nfcn  = 120
a1	  = -162.544	 +/-  68.4829	(limited)
a2	  = 2.68193	 +/-  1.59422	(limited)
mass	  = 124.399	 +/-  0.679381	(limited)
nbkg	  = 57.6493	 +/-  11.7784	(limited)
nsig	  = 65.3599	 +/-  12.0983	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: dea

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -42.62825752 Edm =       20559.18611 NCalls =     21
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : -78.72052167
  Edm           : 27.42260385
  Internal parameters:	[    -0.2013579208     -0.304692654    -0.2799583375    -0.6435011088    -0.6435011088]	
  Internal gradient  :	[   -0.02164934579   -0.02499049059      7.962463555      79.60839205      74.39159701]	
  Internal covariance matrix:
[[      37.266301              0              0              0              0]
 [              0      22.949275              0              0              0]
 [              0              0   0.0019043467              0              0]
 [              0              0              0    0.009355987         

In [8]:
fit_results.Print("v")


  RooFitResult: minimized FCN value: -96.0244, estimated distance to minimum: 0.000434894
                covariance matrix quality: Full matrix, but forced positive-definite
                Status : MINIMIZE=0 HESSE=0 

    Constant Parameter    Value     
  --------------------  ------------
                 alpha    6.0000e-01
                     n    2.0000e+01
                 width    1.7447e+00

    Floating Parameter  InitialValue    FinalValue +/-  Error     GblCorr.
  --------------------  ------------  --------------------------  --------
                    a1   -1.6000e+02   -1.6254e+02 +/-  6.35e+01  <none>
                    a2    2.7000e+00    2.6819e+00 +/-  1.52e+00  <none>
                  mass    1.3000e+02    1.2440e+02 +/-  6.85e-01  <none>
                  nbkg    1.0000e+02    5.7649e+01 +/-  1.17e+01  <none>
                  nsig    1.0000e+02    6.5360e+01 +/-  1.20e+01  <none>



In [9]:
#### Import dataset and model (and automatically all related elements) into a RooWorkspace ####

# create the workspace
w = ROOT.RooWorkspace("w")
w.Import(data)
w.Import(model)

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset dataset
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::m
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooCBShape::smodel
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mass
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::width
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::alpha
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::n
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nsig
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooPolynomial::bmodel
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a2
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) im

In [10]:
# check by printing
w.Print()


RooWorkspace(w) w contents

variables
---------
(a1,a2,alpha,m,mass,n,nbkg,nsig,width)

p.d.f.s
-------
RooPolynomial::bmodel[ x=m coefList=(a1,a2) ] = 26064.4
RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1
RooCBShape::smodel[ m=m m0=mass sigma=width alpha=alpha n=n ] = 1.48809e-06

datasets
--------
RooDataSet::dataset(m)



Define a Model Config "ModelConfig" for the signal plus background model. Use the Higgs signal yield as the parameter of interest, the floating parameters are nuisance, and ensure constants are explicitly set as constant.

This model configuration must be used for:

- The Significance vs Mass plot  
- Hypothesis test (StandardHypoTestDemo)

In [11]:
# Model Configuration: this is for Significance
mc = ROOT.RooStats.ModelConfig("ModelConfig")
mc.SetWorkspace(w)
mc.SetPdf(w.pdf("model"))
mc.SetObservables(w.var("m"))

# set explicit constants 
w.var("width").setConstant(True)
w.var("alpha").setConstant(True)
w.var("n").setConstant(True)

# parameter of interest is the Higgs signal yield
mc.SetParametersOfInterest(w.var("nsig"))

# define a list of nuisance parameters
w.defineSet("nuis", "nbkg,a1,a2")
mc.SetNuisanceParameters(w.set("nuis"))

# special case: set also the mass as a constant. This is required for the Significance vs Mass plot
w.var("mass").setConstant(True)

mc.SetSnapshot(w.var("nsig"))

In [12]:
# Import the ModelConfig for significance into the workspace
w.Import(mc)

False

Define a second ModelConfig "ModelConfig_mass" where

- The parameter of interest is the Higgs Mass
- The set of nuisance parameters includes the signal yield 

In [13]:
# Model Configuration: this is for the Mass intervals
mc2 = ROOT.RooStats.ModelConfig("ModelConfig_Mass")
mc2.SetWorkspace(w)
mc2.SetPdf(w.pdf("model"))
mc2.SetObservables(w.var("m"))

# set explicit constants 
w.var("width").setConstant(True)
w.var("alpha").setConstant(True)
w.var("n").setConstant(True)

# parameter of interest is the Higgs mass
mc2.SetParametersOfInterest(w.var("mass"))

# define a list of nuisance parameters
w.defineSet("nuis2", "nbkg,a1,a2,nsig")
mc2.SetNuisanceParameters(w.set("nuis2"))

# make sure the mass is not constant here
w.var("mass").setConstant(False)

In [14]:
# Import the ModelConfig_mass into the workspace
w.Import(mc2)

False

In [15]:
# check what is inside the workspace
w.Print("v")


RooWorkspace(w) w contents

variables
---------
(a1,a2,alpha,m,mass,n,nbkg,nsig,width)

p.d.f.s
-------
RooPolynomial::bmodel[ x=m coefList=(a1,a2) ] = 26064.4
RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1
RooCBShape::smodel[ m=m m0=mass sigma=width alpha=alpha n=n ] = 1.48809e-06

datasets
--------
RooDataSet::dataset(m)

parameter snapshots
-------------------
ModelConfig__snapshot = (nsig=65.3599 +/- 11.9881)

named sets
----------
ModelConfig_Mass_NuisParams:(nbkg,a1,a2,nsig)
ModelConfig_Mass_Observables:(m)
ModelConfig_Mass_POI:(mass)
ModelConfig_NuisParams:(nbkg,a1,a2)
ModelConfig_Observables:(m)
ModelConfig_POI:(nsig)
ModelConfig__snapshot:(nsig)
nuis:(nbkg,a1,a2)
nuis2:(nbkg,a1,a2,nsig)

generic objects
---------------
RooStats::ModelConfig::ModelConfig
RooStats::ModelConfig::ModelConfig_Mass



In [16]:
# Save the workspace to a file
w.writeToFile("atlas_4l.root")

False

## PART 2: Higgs Mass Profile Likelihood Interval

- Compute a 68% C.L. interval on the Higgs mass, using RooStats Profile Likelihood Calculator.
- Plot the likelihood interval and adjust the range by hand (right click on axis and setrangeuser).  
x from 120 to 128, y from 0 to 20
- Try to change the confidence level.

In [17]:
# create the class using data and ModelConfig_mass
plc = ROOT.RooStats.ProfileLikelihoodCalculator(data, mc2)
# set the confidence level to 68%
plc.SetConfidenceLevel(0.68)
# compute the interval of the parameter of interest, the Higgs Mass
interval = plc.GetInterval()

# print the interval 
lowerlimit = interval.LowerLimit(w.var("mass"))
upperlimit = interval.UpperLimit(w.var("mass"))
print("68% CL interval on Higgs mass: [{:.3f}, {:.3f}] GeV".format(lowerlimit, upperlimit))

# plot the interval
plot = ROOT.RooStats.LikelihoodIntervalPlot(interval)
c2 = ROOT.TCanvas("c2", "c2")
plot.SetRange(120,128)
plot.Draw()
c2.Update()
plot.GetPlottedObject().GetYaxis().SetRangeUser(0, 20)
c2.Draw()

68% CL interval on Higgs mass: [123.768, 125.139] GeV
[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 / Migrad with strategy 1
[#1] INFO:Minimization -- 
  RooFitResult: minimized FCN value: -96.0248, estimated distance to minimum: 8.53002e-10
                covariance matrix quality: Full, accurate covariance matrix
   

In [23]:
# create the class using data and ModelConfig_mass
plc2 = ROOT.RooStats.ProfileLikelihoodCalculator(data, mc2)
# set the confidence level to 90%
plc2.SetConfidenceLevel(0.90)
# compute the interval of the parameter of interest, the Higgs Mass
interval2 = plc2.GetInterval()

# print the interval
lowerlimit2 = interval2.LowerLimit(w.var("mass"))
upperlimit2 = interval2.UpperLimit(w.var("mass"))
print("90% CL interval on Higgs mass: [{:.3f}, {:.3f}] GeV".format(lowerlimit2, upperlimit2))

# plot the interval
plot2 = ROOT.RooStats.LikelihoodIntervalPlot(interval2)
c3 = ROOT.TCanvas("c3", "c3")
plot2.SetRange(120,128)
plot2.Draw()
c3.Update()
plot2.GetPlottedObject().GetYaxis().SetRangeUser(0, 20)
c3.Draw()

90% CL interval on Higgs mass: [123.389, 125.614] GeV
[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 / Migrad with strategy 1
    ----> Doing a re-scan first
[#1] INFO:Minimization -- 
  RooFitResult: minimized FCN value: -96.0248, estimated distance to minimum: 2.17858e-10
                covariance matrix quality: Ful

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3


## PART 3: Higgs mass Feldman Cousin interval

- Compute a 90% C.L interval on the Higgs mass, using RooStats Feldman Cousin Calculator.  
- Try to change the confidence level.

In [26]:
# create the class using data and ModelConfig_mass
fc = ROOT.RooStats.FeldmanCousins(data, mc2)
# set the confidence level to 90%
fc.SetConfidenceLevel(0.90)
# speed it up a bit
fc.UseAdaptiveSampling(True)
# set how many points per parameter of interest to scan
fc.SetNBins(10)
# compute the interval of the parameter of interest, the Higgs mass
fcinterval = fc.GetInterval()
# print the interval (get a pointer of "mass" from mc2)
fclowerlimit = fcinterval.LowerLimit(w.var("mass"))
fcupperlimit = fcinterval.UpperLimit(w.var("mass"))

print("Feldman Cousin 90% CL interval on Higgs mass: [{:.3f}, {:.3f}] GeV".format(fclowerlimit, fcupperlimit))

Feldman Cousin 90% CL interval on Higgs mass: [124.000, 124.000] GeV

=== Using the following for ModelConfig_Mass ===
Observables:             RooArgSet:: = (m)
Parameters of Interest:  RooArgSet:: = (mass)
Nuisance Parameters:     RooArgSet:: = (nbkg,a1,a2,nsig)
PDF:                     RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 55017.1/1

FeldmanCousins: ntoys per point: adaptive
FeldmanCousins: nEvents per toy will fluctuate about  expectation
FeldmanCousins: Model has nuisance parameters, will do profile construction
FeldmanCousins: # points to test = 10
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing

In [29]:
# create the class using data and ModelConfig_mass
fc2 = ROOT.RooStats.FeldmanCousins(data, mc2)
# set the confidence level to 68%
fc2.SetConfidenceLevel(0.68)
# speed it up a bit
fc2.UseAdaptiveSampling(True)
# set how many points per parameter of interest to scan
fc2.SetNBins(10)
# compute the interval of the parameter of interest, the Higgs mass
fcinterval2 = fc2.GetInterval()
# print the interval (get a pointer of "mass" from mc2)
fclowerlimit2 = fcinterval2.LowerLimit(w.var("mass"))
fcupperlimit2 = fcinterval2.UpperLimit(w.var("mass"))

print("Feldman Cousin 68% CL interval on Higgs mass: [{:.3f}, {:.3f}] GeV".format(fclowerlimit2, fcupperlimit2))

Feldman Cousin 68% CL interval on Higgs mass: [124.000, 148.000] GeV

=== Using the following for ModelConfig_Mass ===
Observables:             RooArgSet:: = (m)
Parameters of Interest:  RooArgSet:: = (mass)
Nuisance Parameters:     RooArgSet:: = (nbkg,a1,a2,nsig)
PDF:                     RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 54390.2/1

FeldmanCousins: ntoys per point: adaptive
FeldmanCousins: nEvents per toy will fluctuate about  expectation
FeldmanCousins: Model has nuisance parameters, will do profile construction
FeldmanCousins: # points to test = 10
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 1/10 total MC = 12 this test stat = 21.9416
 mass=112

## PART 4: Compute the significance (Hypothesis Test)

- To estimate significance, we perform a hypothesis test to reject the null (background-only) model in favor of the alternate (signal-plus-background) model. In RooStats, this involves defining two ModelConfig objects: one for the null and one for the alternate model.

- The null hypothesis (i.e. the peak in the data is just a statistical fluctuation of the background only) corresponds to a Higgs signal yield nsignal = 0.

In [30]:
# create the class using data and ModelConfig for the Higgs signal
plc4 = ROOT.RooStats.ProfileLikelihoodCalculator(data, mc)
w.var("nsig").setVal(0)
plc4.SetNullParameters(w.var("nsig"))
hypotest = plc4.GetHypoTest()
alpha = hypotest.NullPValue()
significance = hypotest.Significance()

print(f"Significance of the Higgs signal: {significance:.3f} \n")

Significance of the Higgs signal: 6.624 

[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 / Migrad with strategy 1
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
[#1] INFO:Minimization -- 
  RooFitResult: minimized FCN value: -96.0248, estimated distance to minimum: 4.03092e

### Method 3: using the ready-to-use RooStats [Standard Demo tutorials](https://root.cern.ch/doc/master/group__tutorial__roostats.html)

#### SIGNIFICANCE (2 = asymptotic calculator, 3 = Profile Likelihood Ratio one-side)  
```js
(base) rianashaba@Rianas-MacBook-Pro ~ % eval "$(/Users/rianashaba/miniforge3/bin/conda shell.zsh hook)"
(base) rianashaba@Rianas-MacBook-Pro ~ % cd ROOT
(base) rianashaba@Rianas-MacBook-Pro ROOT % conda activate root_env
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % ls  
CountingModel.root		higgs_4l.dat
Exercise11_Ad.ipynb		minos_2013_data.dat
Exercise12_2.ipynb		minos_2013_mc.dat
Exercise13_1.ipynb		minos_data.png
Exercise13_2.ipynb		rarest_b0_decay.dat
Exercise2021_1.ipynb		sess1.ipynb
Exercise2021_2.ipynb		session1.ipynb
ExerciseA.ipynb			session2_1.ipynb
ExerciseATLAS.ipynb		session2_2.ipynb
ExerciseOPERA.ipynb		session4_1.ipynb
StandardFeldmanCousinDemo.C	test.root
atlas_4l.root			test.rtf
b0_invariant_mass.root		tutorials
exercise1.ipynb
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % root atlas_4l.root 
   ------------------------------------------------------------------
  | Welcome to ROOT 6.34.04                        https://root.cern |
  | (c) 1995-2024, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for macosxarm64 on Feb 26 2025, 15:56:26                   |
  | From tags/6-34-04@6-34-04                                        |
  | With                                                             |
  | Try '.help'/'.?', '.demo', '.license', '.credits', '.quit'/'.q'  |
   ------------------------------------------------------------------

root [0] 
Attaching file atlas_4l.root as _file0...
(TFile *) 0x149012160
root [1] .ls
TFile**		atlas_4l.root	
 TFile*		atlas_4l.root	
  KEY: RooWorkspace	w;1	w
  KEY: TProcessID	ProcessID0;1	0658d63c-d436-11f0-9ac5-020a14acbeef
root [2] w
(RooWorkspace *) 0x149c96c00
root [3] w->Print()

RooWorkspace(w) w contents

variables
---------
(a1,a2,alpha,m,mass,n,nbkg,nsig,width)

p.d.f.s
-------
RooPolynomial::bmodel[ x=m coefList=(a1,a2) ] = 26064.4
RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1
RooCBShape::smodel[ m=m m0=mass sigma=width alpha=alpha n=n ] = 1.48809e-06

datasets
--------
RooDataSet::dataset(m)

parameter snapshots
-------------------
ModelConfig__snapshot = (nsig=65.3599 +/- 11.9881)

named sets
----------
ModelConfig_Mass_NuisParams:(nbkg,a1,a2,nsig,width)
ModelConfig_Mass_Observables:(m)
ModelConfig_Mass_POI:(mass)
ModelConfig_NuisParams:(nbkg,a1,a2)
ModelConfig_Observables:(m)
ModelConfig_POI:(nsig)
ModelConfig__snapshot:(nsig)
nuis:(nbkg,a1,a2)
nuis2:(nbkg,a1,a2,nsig,width)

generic objects
---------------
RooStats::ModelConfig::ModelConfig
RooStats::ModelConfig::ModelConfig_Mass

root [4] .L $ROOTSYS/tutorials/roostats/StandardHypoTestDemo.C 
root [5] StandardHypoTestDemo(
void StandardHypoTestDemo(const char* infile = "", const char* workspaceName = "combined", const char* modelSBName = "ModelConfig", const char* modelBName = "", const char* dataName = "obsData", int calcType = 0, int testStatType = 3, int ntoys = 5000, bool useNC = false, const char* nuisPriorName = 0)
root [5] StandardHypoTestDemo("/Users/rianashaba/ROOT/atlas_4l.root", "w", "ModelConfig", "", "dataset", 2, 3)

RooWorkspace(w) w contents

variables
---------
(a1,a2,alpha,m,mass,n,nbkg,nsig,width)

p.d.f.s
-------
RooPolynomial::bmodel[ x=m coefList=(a1,a2) ] = 26064.4
RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1
RooCBShape::smodel[ m=m m0=mass sigma=width alpha=alpha n=n ] = 1.48809e-06

datasets
--------
RooDataSet::dataset(m)

parameter snapshots
-------------------
ModelConfig__snapshot = (nsig=65.3599 +/- 11.9881)

named sets
----------
ModelConfig_Mass_NuisParams:(nbkg,a1,a2,nsig,width)
ModelConfig_Mass_Observables:(m)
ModelConfig_Mass_POI:(mass)
ModelConfig_NuisParams:(nbkg,a1,a2)
ModelConfig_Observables:(m)
ModelConfig_POI:(nsig)
ModelConfig__snapshot:(nsig)
nuis:(nbkg,a1,a2)
nuis2:(nbkg,a1,a2,nsig,width)

generic objects
---------------
RooStats::ModelConfig::ModelConfig
RooStats::ModelConfig::ModelConfig_Mass

Info in <StandardHypoTestInvDemo>: The background model  does not exist
Info in <StandardHypoTestInvDemo>: Copy it from ModelConfig ModelConfig and set POI to zero
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize - Find  best unconditional NLL on observed data
[#0] PROGRESS:Eval -- Best fitted POI value = 65.373 +/- 12.1007
[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set
[#1] INFO:InputArguments -- AsymptoticCalculator: Asimov data will be generated using fitted nuisance parameter values
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize Find  best conditional NLL on ASIMOV data set for given alt POI ( nsig ) = 65.3599
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> trying with improve

Results HypoTestAsymptotic_result: 
 - Null p-value = 1.74341e-11
 - Significance = 6.62439
 - CL_b: 1.74341e-11
 - CL_s+b: 0.675022
 - CL_s: 3.87186e+10
Asymptotic results 
 Expected p -value and significance at -2 sigma = 1.90498e-07 significance 5.07822 sigma 
 Expected p -value and significance at -1 sigma = 6.07636e-10 significance 6.07822 sigma 
 Expected p -value and significance at 0 sigma = 7.30111e-13 significance 7.07822 sigma 
 Expected p -value and significance at 1 sigma = 3.28606e-16 significance 8.07822 sigma 
 Expected p -value and significance at 2 sigma = 5.51846e-20 significance 9.07822 sigma 
root [6] 
```

#### 68% C.L. Profile Likelihood Interval 

```js
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % root atlas_4l.root 
   ------------------------------------------------------------------
  | Welcome to ROOT 6.34.04                        https://root.cern |
  | (c) 1995-2024, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for macosxarm64 on Feb 26 2025, 15:56:26                   |
  | From tags/6-34-04@6-34-04                                        |
  | With                                                             |
  | Try '.help'/'.?', '.demo', '.license', '.credits', '.quit'/'.q'  |
   ------------------------------------------------------------------

root [0] 
Attaching file atlas_4l.root as _file0...
(TFile *) 0x127e99230
root [1] .L $ROOTSYS/tutorials/roostats/StandardProfileLikelihoodDemo.C
root [2] optPL.confLevel = 0.68
(double) 0.68000000
root [3] StandardProfileLikelihoodDemo(
void StandardProfileLikelihoodDemo(const char* infile = "", const char* workspaceName = "combined", const char* modelConfigName = "ModelConfig", const char* dataName = "obsData")
root [3] StandardProfileLikelihoodDemo("/Users/rianashaba/ROOT/atlas_4l.root", "w", "ModelConfig_Mass", "dataset")
[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 /  with strategy 1
[#1] INFO:Minimization -- 
  RooFitResult: minimized FCN value: -96.0248, estimated distance to minimum: 8.53002e-10
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                    a1   -1.5897e+02 +/-  5.94e+01
                    a2    2.7408e+00 +/-  1.54e+00
                  mass    1.2441e+02 +/-  6.79e-01
                  nbkg    5.7536e+01 +/-  1.18e+01
                  nsig    6.5464e+01 +/-  1.21e+01


>>>> RESULT : 68% interval on mass is : [123.768, 125.139]
 
making a plot of the profile likelihood function ....(if it is taking a lot of time use less points or the TF1 drawing option)
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) minimum found at (mass=124.431)
.
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) determining minimum likelihood for current configurations w.r.t all observable
[#0] ERROR:InputArguments -- RooArgSet::checkForDup: ERROR argument with name mass is already in this set
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) minimum found at (mass=124.418)
..........................................................................................................................................................................................Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
root [4] Info in <TCanvas::Print>: pdf file /Users/rianashaba/ROOT/plcatlas.pdf has been created   
```

#### 90% C.L. Profile Likelihood Interval  
```js
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % root atlas_4l.root
   ------------------------------------------------------------------
  | Welcome to ROOT 6.34.04                        https://root.cern |
  | (c) 1995-2024, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for macosxarm64 on Feb 26 2025, 15:56:26                   |
  | From tags/6-34-04@6-34-04                                        |
  | With                                                             |
  | Try '.help'/'.?', '.demo', '.license', '.credits', '.quit'/'.q'  |
   ------------------------------------------------------------------

root [0] 
Attaching file atlas_4l.root as _file0...
(TFile *) 0x14af99270
root [1] .L $ROOTSYS/tutorials/roostats/StandardProfileLikelihoodDemo.C
root [2] optPL.confLevel = 0.90
(double) 0.90000000
root [3] StandardProfileLikelihoodDemo(
void StandardProfileLikelihoodDemo(const char* infile = "", const char* workspaceName = "combined", const char* modelConfigName = "ModelConfig", const char* dataName = "obsData")
root [3] StandardProfileLikelihoodDemo("/Users/rianashaba/ROOT/atlas_4l.root", "w", "ModelConfig_Mass", "dataset")
[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 /  with strategy 1
[#1] INFO:Minimization -- 
  RooFitResult: minimized FCN value: -96.0248, estimated distance to minimum: 8.53002e-10
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                    a1   -1.5897e+02 +/-  5.94e+01
                    a2    2.7408e+00 +/-  1.54e+00
                  mass    1.2441e+02 +/-  6.79e-01
                  nbkg    5.7536e+01 +/-  1.18e+01
                  nsig    6.5464e+01 +/-  1.21e+01


>>>> RESULT : 90% interval on mass is : [123.389, 125.614]
 
making a plot of the profile likelihood function ....(if it is taking a lot of time use less points or the TF1 drawing option)
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) minimum found at (mass=124.425)
.
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) determining minimum likelihood for current configurations w.r.t all observable
[#0] ERROR:InputArguments -- RooArgSet::checkForDup: ERROR argument with name mass is already in this set
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[mass]) minimum found at (mass=124.416)
........................................................................................................................................................................................Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
root [4] 
```

#### 90% C.L. Feldman Cousins Interval  
```js
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % root atlas_4l.root
   ------------------------------------------------------------------
  | Welcome to ROOT 6.34.04                        https://root.cern |
  | (c) 1995-2024, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for macosxarm64 on Feb 26 2025, 15:56:26                   |
  | From tags/6-34-04@6-34-04                                        |
  | With                                                             |
  | Try '.help'/'.?', '.demo', '.license', '.credits', '.quit'/'.q'  |
   ------------------------------------------------------------------

root [0] 
Attaching file atlas_4l.root as _file0...
(TFile *) 0x12f657dc0
root [1] .L StandardFeldmanCousinDemo.C 
root [2] StandardFeldmanCousinsDemo(
void StandardFeldmanCousinsDemo(const char* infile = "", const char* workspaceName = "combined", const char* modelConfigName = "ModelConfig", const char* dataName = "obsData")
root [2] StandardFeldmanCousinsDemo("/Users/rianashaba/ROOT/atlas_4l.root", "w", "ModelConfig_Mass", "dataset")

=== Using the following for ModelConfig_Mass ===
Observables:             RooArgSet:: = (m)
Parameters of Interest:  RooArgSet:: = (mass)
Nuisance Parameters:     RooArgSet:: = (nbkg,a1,a2,nsig)
PDF:                     RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1

FeldmanCousins: ntoys per point: adaptive
FeldmanCousins: nEvents per toy will fluctuate about  expectation
FeldmanCousins: Model has nuisance parameters, will do profile construction
FeldmanCousins: # points to test = 10
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
lookup index = 0
NeymanConstruction: Prog: 1/10 total MC = 40 this test stat = 21.9415
 mass=112 nbkg=123.205 a1=-199.729 a2=3.30468 nsig=0.000198026 [-inf, 2.24042]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 2/10 total MC = 40 this test stat = 21.9414
 mass=116 nbkg=122.928 a1=-139.435 a2=2.3531 nsig=0.000205344 [-inf, 1.49145]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
NeymanConstruction: Prog: 3/10 total MC = 40 this test stat = -1
 mass=120 nbkg=106.758 a1=-187.23 a2=2.06042 nsig=16.2669 [-inf, 1.84451]  in interval = 1

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
NeymanConstruction: Prog: 4/10 total MC = 40 this test stat = 0.199363
 mass=124 nbkg=59.8312 a1=-199.762 a2=2.58189 nsig=63.1883 [-inf, 1.79087]  in interval = 1

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
NeymanConstruction: Prog: 5/10 total MC = 40 this test stat = 14.3351
 mass=128 nbkg=79.9277 a1=-100 a2=3.99621 nsig=43.1736 [-inf, 0.701291]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
NeymanConstruction: Prog: 6/10 total MC = 40 this test stat = 21.9825
 mass=132 nbkg=122.852 a1=-100.024 a2=3.70222 nsig=1.9802e-07 [-inf, 2.4396]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 7/10 total MC = 40 this test stat = 21.9848
 mass=136 nbkg=122.996 a1=-100.011 a2=3.96658 nsig=4.81334e-06 [-inf, 3.67927]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
NeymanConstruction: Prog: 8/10 total MC = 40 this test stat = 21.984
 mass=140 nbkg=123.027 a1=-100 a2=3.92848 nsig=2.87648e-06 [-inf, 2.15823]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 9/10 total MC = 40 this test stat = 21.9842
 mass=144 nbkg=122.977 a1=-100.073 a2=3.89845 nsig=1.64081e-05 [-inf, 2.33018]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 10/10 total MC = 40 this test stat = -1
 mass=148 nbkg=123.006 a1=-100.012 a2=3.84078 nsig=1.12666e-05 [-inf, 2.45062]  in interval = 1

[#1] INFO:Eval -- 3 points in interval

90% interval on mass is : [120, 148] 
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
root [3]
```

#### 68% C.L. Feldman Cousins Interval 
```js
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % root atlas_4l.root
   ------------------------------------------------------------------
  | Welcome to ROOT 6.34.04                        https://root.cern |
  | (c) 1995-2024, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for macosxarm64 on Feb 26 2025, 15:56:26                   |
  | From tags/6-34-04@6-34-04                                        |
  | With                                                             |
  | Try '.help'/'.?', '.demo', '.license', '.credits', '.quit'/'.q'  |
   ------------------------------------------------------------------

root [0] 
Attaching file atlas_4l.root as _file0...
(TFile *) 0x149055300
root [1] .L StandardFeldmanCousinDemo.C
root [2] StandardFeldmanCousinsDemo(
void StandardFeldmanCousinsDemo(const char* infile = "", const char* workspaceName = "combined", const char* modelConfigName = "ModelConfig", const char* dataName = "obsData")
root [2] StandardFeldmanCousinsDemo("/Users/rianashaba/ROOT/atlas_4l.root", "w", "ModelConfig_Mass", "dataset")

=== Using the following for ModelConfig_Mass ===
Observables:             RooArgSet:: = (m)
Parameters of Interest:  RooArgSet:: = (mass)
Nuisance Parameters:     RooArgSet:: = (nbkg,a1,a2,nsig)
PDF:                     RooAddPdf::model[ nsig * smodel + nbkg * bmodel ] = 12215.3/1

FeldmanCousins: ntoys per point: adaptive
FeldmanCousins: nEvents per toy will fluctuate about  expectation
FeldmanCousins: Model has nuisance parameters, will do profile construction
FeldmanCousins: # points to test = 10
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
lookup index = 0
NeymanConstruction: Prog: 1/10 total MC = 12 this test stat = 21.9415
 mass=112 nbkg=123.205 a1=-199.729 a2=3.30468 nsig=0.000198026 [-inf, 1.44403]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 2/10 total MC = 12 this test stat = 21.9414
 mass=116 nbkg=122.928 a1=-139.435 a2=2.3531 nsig=0.000205344 [-inf, 2.68893]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
NeymanConstruction: Prog: 3/10 total MC = 12 this test stat = -1
 mass=120 nbkg=106.758 a1=-187.23 a2=2.06042 nsig=16.2669 [-inf, 0.830274]  in interval = 1

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
NeymanConstruction: Prog: 4/10 total MC = 108 this test stat = 0.199363
 mass=124 nbkg=59.8312 a1=-199.762 a2=2.58189 nsig=63.1883 [-inf, 0.458607]  in interval = 1

    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 5/10 total MC = 12 this test stat = 14.3351
 mass=128 nbkg=79.9277 a1=-100 a2=3.99621 nsig=43.1736 [-inf, 0.971939]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 6/10 total MC = 12 this test stat = 21.9825
 mass=132 nbkg=122.852 a1=-100.024 a2=3.70222 nsig=1.9802e-07 [-inf, 1.14338]  in interval = 0

    ----> Doing a re-scan first
NeymanConstruction: Prog: 7/10 total MC = 12 this test stat = 21.9848
 mass=136 nbkg=122.996 a1=-100.011 a2=3.96658 nsig=4.81334e-06 [-inf, 0.0800852]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 8/10 total MC = 12 this test stat = 21.984
 mass=140 nbkg=123.027 a1=-100 a2=3.92848 nsig=2.87648e-06 [-inf, 0.291648]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 9/10 total MC = 12 this test stat = 21.9842
 mass=144 nbkg=122.977 a1=-100.073 a2=3.89845 nsig=1.64081e-05 [-inf, 1.18733]  in interval = 0

    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
    ----> trying with improve
    ----> Doing a re-scan first
    ----> Doing a re-scan first
NeymanConstruction: Prog: 10/10 total MC = 12 this test stat = -1
 mass=148 nbkg=123.006 a1=-100.012 a2=3.84078 nsig=1.12666e-05 [-inf, 1.03002]  in interval = 1

[#1] INFO:Eval -- 3 points in interval

68% interval on mass is : [120, 148] 
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
root [3]
```